## Aplicación de los Algoritmos de Ordenamiento


In [ ]:
import os
import re
import time
import random
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import heapq
from functools import cmp_to_key
from collections import defaultdict
import math
import bisect

from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Definir rutas
BIBTEX_PATH =os.getenv("CONSOLIDADO_PATH")
OUTPUT_PATH = os.getenv("ORDENAMIENTO_PATH")
TIMING_PATH = os.getenv("TIEMPOS_PATH")

# Crear directorios si no existen
Path(OUTPUT_PATH).mkdir(parents=True, exist_ok=True)
Path(TIMING_PATH).mkdir(parents=True, exist_ok=True)

# %%
def extract_bibtex_fields(file_path):
    """
    Extrae los campos de un archivo BibTeX.
    Se buscan las claves: author, title, journal, year, doi, keywords y abstract.
    Para journal se considera también booktitle.
    Si un campo no existe se asigna como cadena vacía.
    """
    fields = {
        'author': [],
        'title': [],
        'journal': [],
        'year': [],
        'doi': [],
        'keywords': [],
        'abstract': []
    }
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Buscar entradas BibTeX (cualquier tipo: article, inproceedings, etc.)
        entries = re.findall(r'@\w+\s*\{[^@]*\}', content, re.DOTALL)
        
        for entry in entries:
            for field in fields.keys():
                # Buscar el campo en la entrada, manejando {valor} o "valor"
                pattern = rf'{field}\s*=\s*[\{{\"](.*?)[\}}\"]'
                match = re.search(pattern, entry, re.IGNORECASE | re.DOTALL)
                
                # Para journal, buscar booktitle si no se encontró journal
                if field == 'journal' and not match:
                    pattern = r'booktitle\s*=\s*[\{{\"](.*?)[\}}\"]'
                    match = re.search(pattern, entry, re.IGNORECASE | re.DOTALL)
                
                if match:
                    value = match.group(1).strip()
                    fields[field].append(value)
                else:
                    fields[field].append("")
        
        return fields
        
    except Exception as e:
        print(f"Error al procesar el archivo BibTeX: {e}")
        return fields

# %%
# Implementación de Algoritmos de Ordenamiento

# 1. TimSort (usando sorted de Python, que es TimSort)
def timsort(arr):
    return sorted(arr)

# 2. Comb Sort
def combsort(arr):
    arr = arr.copy()
    n = len(arr)
    gap = n
    shrink = 1.3
    sorted_flag = False
    
    while not sorted_flag:
        gap = int(gap / shrink)
        if gap <= 1:
            gap = 1
            sorted_flag = True
        
        i = 0
        while i + gap < n:
            if arr[i] > arr[i + gap]:
                arr[i], arr[i + gap] = arr[i + gap], arr[i]
                sorted_flag = False
            i += 1
    
    return arr

# 3. Selection Sort
def selectionsort(arr):
    arr = arr.copy()
    n = len(arr)
    
    for i in range(n):
        min_idx = i
        for j in range(i+1, n):
            if arr[j] < arr[min_idx]:
                min_idx = j
        arr[i], arr[min_idx] = arr[min_idx], arr[i]
    
    return arr

# 4. Tree Sort (con eliminación de duplicados antes de insertar)
class Nodo:
    def __init__(self, clave):
        self.clave = clave
        self.izq = None
        self.der = None

def insertar(raiz, clave):
    if raiz is None:
        return Nodo(clave)
    if clave < raiz.clave:
        raiz.izq = insertar(raiz.izq, clave)
    else:
        raiz.der = insertar(raiz.der, clave)
    return raiz

def inorden(raiz, resultado):
    if raiz is not None:
        inorden(raiz.izq, resultado)
        resultado.append(raiz.clave)
        inorden(raiz.der, resultado)

def treesort(arr):
    if not arr:
        return []
    raiz = None
    # 🔑 usamos set() para eliminar duplicados ANTES de insertar
    for item in set(arr):
        clave = str(item).strip().lower()
        raiz = insertar(raiz, clave)
    resultado = []
    inorden(raiz, resultado)
    return resultado


# 5. Pigeonhole Sort
def pigeonholesort(arr):
    if not arr:
        return []
    
    min_val = 0
    max_val = 0
    
    str_to_num = {}
    for i, s in enumerate(arr):
        if s == "":
            val = -1  # Manejar cadenas vacías
        else:
            val = hash(s) % 1000000  # Limitar el rango
        str_to_num[s] = val
        
        if i == 0 or val < min_val:
            min_val = val
        if i == 0 or val > max_val:
            max_val = val
    
    size = max_val - min_val + 1
    holes = [[] for _ in range(size)]
    
    for s in arr:
        val = str_to_num[s]
        holes[val - min_val].append(s)
    
    result = []
    for hole in holes:
        result.extend(sorted(hole))
    
    return result

# 6. Bucket Sort
def bucketsort(arr):
    if not arr:
        return []
    
    buckets = defaultdict(list)
    
    for s in arr:
        key = s[0].lower() if s != "" else ""
        buckets[key].append(s)
    
    result = []
    for key in sorted(buckets.keys()):
        result.extend(sorted(buckets[key]))
    
    return result

# 7. Quick Sort
def quicksort(arr):
    arr = arr.copy()
    
    def _quicksort(arr, low, high):
        if low < high:
            pi = partition(arr, low, high)
            _quicksort(arr, low, pi - 1)
            _quicksort(arr, pi + 1, high)
    
    def partition(arr, low, high):
        pivot = arr[high]
        i = low - 1
        
        for j in range(low, high):
            if arr[j] <= pivot:
                i += 1
                arr[i], arr[j] = arr[j], arr[i]
        arr[i + 1], arr[high] = arr[high], arr[i + 1]
        return i + 1
    
    _quicksort(arr, 0, len(arr) - 1)
    return arr

# 8. Heap Sort
def heapsort(arr):
    heap = arr.copy()
    heapq.heapify(heap)
    result = []
    while heap:
        result.append(heapq.heappop(heap))
    return result

# 9. Gnome Sort
def gnomesort(arr):
    arr = arr.copy()
    n = len(arr)
    pos = 0
    
    while pos < n:
        if pos == 0 or arr[pos] >= arr[pos - 1]:
            pos += 1
        else:
            arr[pos], arr[pos - 1] = arr[pos - 1], arr[pos]
            pos -= 1
    
    return arr

# 10. Binary Insertion Sort
def binaryInsertionsort(arr):
    arr = arr.copy()
    for i in range(1, len(arr)):
        key = arr[i]
        lo, hi = 0, i
        while lo < hi:
            mid = (lo + hi) // 2
            if arr[mid] <= key:
                lo = mid + 1
            else:
                hi = mid
        arr.insert(lo, arr.pop(i))
    return arr

# 11. Radix Sort para strings (LSD)
def radixsort(arr):
    if not arr:
        return []
    # Encontrar la longitud máxima de las cadenas
    max_len = max(len(s) for s in arr)
    # Procesar desde el último carácter hasta el primero
    for pos in range(max_len - 1, -1, -1):
        buckets = {}
        for s in arr:
            # Si la cadena es muy corta, usamos "" (que se ordena antes)
            key = s[pos] if pos < len(s) else ""
            buckets.setdefault(key, []).append(s)
        sorted_keys = sorted(buckets.keys())
        arr = []
        for key in sorted_keys:
            arr.extend(buckets[key])
    return arr

# %%
# Diccionario de algoritmos a utilizar
algorithms = {
    'timsort': timsort,
    'combsort': combsort,
    'selectionsort': selectionsort,
    'treesort': treesort,
    'pigeonholesort': pigeonholesort,
    'bucketsort': bucketsort,
    'quicksort': quicksort,
    'heapsort': heapsort,
    'gnomesort': gnomesort,
    'binaryInsertionsort': binaryInsertionsort,
    'radixsort': radixsort
}

# %%
# Extraer campos del archivo BibTeX
fields = extract_bibtex_fields(BIBTEX_PATH)

# Diccionarios para almacenar resultados ordenados y tiempos
sorted_results = { key: {} for key in fields.keys() }
timings = { key: {} for key in fields.keys() }

# Para cada campo, aplicar cada algoritmo y medir el tiempo de ejecución
import random

for field, values in fields.items():
    # Saltar campos vacíos
    if not values:
        print(f"⚠️ El campo '{field}' está vacío, omitiendo...")
        continue
        
    for algo_name, algo_func in algorithms.items():
        try:
            # Hacer una copia de values para no modificar el original
            values_to_sort = values.copy()
            start_time = time.time()
            sorted_list = algo_func(values_to_sort)
            end_time = time.time()
            elapsed = (end_time - start_time)
            
            sorted_results[field][algo_name] = sorted_list
            timings[field][algo_name] = elapsed
        except Exception as e:
            print(f"⚠️ Error en {algo_name} con el campo {field}: {str(e)}")
            sorted_results[field][algo_name] = []
            timings[field][algo_name] = float("nan")

# %%
# Escribir archivos de salida para cada clave en OUTPUT_PATH
for field, algo_results in sorted_results.items():
    file_path = os.path.join(OUTPUT_PATH, f"{field}.txt")
    with open(file_path, 'w', encoding='utf-8') as f:
        for algo_name in algorithms.keys():
            f.write(f"# ordenamiento por {algo_name}\n")
            for item in algo_results[algo_name]:
                f.write(item + "\n")
            f.write("\n")

# %%
# Escribir archivo con tiempos de ordenamiento en TIMING_PATH
timing_file = os.path.join(TIMING_PATH, "tiempo-ordenamiento.txt")
with open(timing_file, 'w', encoding='utf-8') as f:
    f.write("Campo,Algoritmo,Tiempo (segundos)\n")
    for field, algo_times in timings.items():
        for algo_name, t in algo_times.items():
            f.write(f"{field},{algo_name},{t}\n")

# %% 
# Crear tabla con tamaño y tiempo de cada algoritmo por campo
resultados_tabla = []

for field, algo_times in timings.items():
    tamaño = len(fields[field])  # número de elementos de ese campo
    for algo_name, t in algo_times.items():
        resultados_tabla.append({
            "Campo": field,
            "Algoritmo": algo_name,
            "Tamaño": tamaño,
            "Tiempo (s)": round(t, 6)
        })

# Convertir a DataFrame
df_resultados = pd.DataFrame(resultados_tabla)

# Mostrar tabla en pantalla
print("\n📊 Resultados de ejecución:\n")
print(df_resultados.to_string(index=False))

# Exportar a CSV y Markdown
csv_path = os.path.join(TIMING_PATH, "tabla_resultados.csv")
md_path = os.path.join(TIMING_PATH, "tabla_resultados.md")

df_resultados.to_csv(csv_path, index=False)

with open(md_path, "w", encoding="utf-8") as f:
    f.write(df_resultados.to_markdown(index=False))

print(f"\nTablas exportadas a:\n- {csv_path}\n- {md_path}")


# %%
# Graficar tiempos de ordenamiento para cada clave
for field, algo_times in timings.items():
    algo_names = list(algo_times.keys())
    times_list = [algo_times[algo] for algo in algo_names]
    
    plt.figure(figsize=(10,6))
    plt.bar(algo_names, times_list)
    plt.xlabel('Algoritmo')
    plt.ylabel('Tiempo (segundos)')
    plt.title(f'Tiempos de ordenamiento para el campo: {field}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    # Guardar la gráfica en TIMING_PATH (por ejemplo, "tiempo_author.png")
    plt.savefig(os.path.join(TIMING_PATH, f"tiempo_{field}.png"))
    plt.show()

## Top 15 Autores mas repetidos


In [ ]:
from collections import Counter
import matplotlib.pyplot as plt
import re

# %% Extraer campos del archivo BibTeX
fields = extract_bibtex_fields(BIBTEX_PATH)

# Separar autores (si vienen en formato "Autor1 and Autor2")
all_authors = []
for entry in fields['author']:
    autores = re.split(r'\s+and\s+', entry, flags=re.IGNORECASE)
    all_authors.extend([a.strip() for a in autores if a.strip() != ""])

# Contar ocurrencias
contador = Counter(all_authors)
top15 = contador.most_common(15)  # ya viene ordenado de mayor a menor

# Para gráfica, invertimos para que el mayor quede arriba en barh
autores, conteos = zip(*reversed(top15))

plt.figure(figsize=(12,6))
plt.barh(autores, conteos)  # gráfica horizontal
plt.xlabel("Número de apariciones")
plt.ylabel("Autores")
plt.title("Top 15 autores con más apariciones")
plt.tight_layout()
plt.show()
